In [9]:
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import lyricsgenius
from langdetect import detect, LangDetectException

import time

In [10]:
# Spotify 认证
client_id = 'f5cc3fbacef443ee942ff4f046fca5ae'
client_secret = 'f984494819074286a0f78dac54e58c2d'

In [11]:
# 初始化 Spotify 客户端
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [14]:
# Genius API 凭证
GENIUS_API_TOKEN = "OJo5CwYvCbiC6-77ZkfurOKcJtkwACznbRkjffdCfpG3phG4TYnID0gnPIrQuQ49"

In [15]:
# 初始化 Genius 客户端
genius_api = lyricsgenius.Genius(GENIUS_API_TOKEN, timeout=60)

# 用于存储歌曲信息的列表
lyrics_data = []

# 设置参数
max_lyrics = 1000  # 最大歌词数量
batch_size = 50  # 每次从 Spotify 分批获取的歌曲数量(limit) 
 
（已分别放进spotify、genius部分）

# Spotify

In [12]:
# 定义搜索参数组合（示例：按年份分批次）
search_queries = [
    "year:2005-2014 genre:pop",   # 2005-2014年的流行歌曲
    "year:2015-2024 genre:pop"   # 2015-2024 年的流行歌曲
]

# 存储所有歌曲的唯一标识符（防止重复）
seen_tracks = set()    # set 集合：存储已处理歌曲，如果是新歌曲则添加到 seen_tracks（检查唯一标识符（如ID）是否已存在,存在则跳过）
all_tracks = []        # 同时将歌曲信息（如title、artist等）添加到 all_tracks

# 分批次搜索
for query in search_queries:
    
    offset = 0       # 开始页数
    batch_size = 50  # 每次从 Spotify 分批获取 50 首 (limit)
    
    while True:
        try:
            results = sp.search(q=query, type="track", limit=batch_size, offset=offset)
            tracks = results["tracks"]["items"]
            
            if not tracks:
                break  # 没有更多结果
            
            for track in tracks:
                track_id = track["id"]
                if track_id not in seen_tracks:
                    seen_tracks.add(track_id)
                    all_tracks.append({
                        "id": track_id,
                        "title": track["name"],
                        "artist": track["artists"][0]["name"],
                        "year": track["album"]["release_date"][:4] if track["album"]["release_date"] else None
                    })
            
            offset += batch_size
            if offset + batch_size > 1000:
                break  # 避免触发 offset + limit > 1000 的错误
            
            time.sleep(1)  # 避免速率限制
            
        except Exception as e:
            print(f"Error for query '{query}': {e}")
            break

In [13]:
# 转换为 DataFrame
df_spotify = pd.DataFrame(all_tracks)
print(f"Total tracks collected: {len(df_spotify)}")

Total tracks collected: 1928


# Genius

In [16]:
# 存储歌词数据
lyrics_data = []

# # 初始化计数器
# lyrics_count = 0

# # 设置最大歌词数量
# max_lyrics = 1000  

In [17]:
# 遍历 Spotify 歌曲列表
for idx, row in df_spotify.iterrows():
    title = row["title"]
    artist = row["artist"]

    # # 如果已经获取到1000首歌词，停止循环
    # if lyrics_count >= max_lyrics:
    #     print(f"Reached the limit of {max_lyrics} lyrics. Stopping.")
    #     break
    
    # 重试机制
    retries = 3
    for attempt in range(retries):       
        
        try:
            song = genius_api.search_song(title, artist)
            if not song:
                print(f"Lyrics not found: {title} by {artist}")
                break
            
            # 检测歌词语言
            try:
                language = detect(song.lyrics)
                if language != "en":
                    print(f"Skipping non-English song: {title} by {artist}")
                    break
            except LangDetectException:
                print(f"Language detection failed: {title} by {artist}")
                break
            
            # 保存歌词
            lyrics_data.append({
                "title": title,
                "artist": artist,
                "lyrics": song.lyrics
            })
            
            # 更新计数器
            lyrics_count += 1  
            
            print(f"Added lyrics: {title} by {artist}")
            break
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {title} by {artist}: {e}")
            time.sleep(2)
            
    else:
        print(f"Failed to get lyrics after {retries} attempts: {title} by {artist}")


# 合并歌词数据
df_lyrics = pd.DataFrame(lyrics_data)

Searching for "Just the Way You Are" by Bruno Mars...
Done.
Added lyrics: Just the Way You Are by Bruno Mars
Searching for "Night Changes" by One Direction...
Done.
Added lyrics: Night Changes by One Direction
Searching for "愛我還是他" by David Tao...
No results found for: '愛我還是他 David Tao'
Lyrics not found: 愛我還是他 by David Tao
Searching for "Talking to the Moon" by Bruno Mars...
Done.
Added lyrics: Talking to the Moon by Bruno Mars
Searching for "說好的幸福呢" by Jay Chou...
Done.
Skipping non-English song: 說好的幸福呢 by Jay Chou
Searching for "楓" by Jay Chou...
Done.
Skipping non-English song: 楓 by Jay Chou
Searching for "一路向北" by Jay Chou...
Done.
Skipping non-English song: 一路向北 by Jay Chou
Searching for "Viva La Vida" by Coldplay...
Done.
Added lyrics: Viva La Vida by Coldplay
Searching for "珊瑚海" by Jay Chou...
Done.
Skipping non-English song: 珊瑚海 by Jay Chou
Searching for "CROOKED" by G-DRAGON...
Done.
Skipping non-English song: CROOKED by G-DRAGON
Searching for "Payphone" by Maroon 5...
Done.
A

In [18]:
# 合并数据（根据标题和艺术家）
df_final = pd.merge(
    df_spotify,
    df_lyrics,
    on=["title", "artist"],
    how="inner"  # 只保留有歌词的记录
)

In [19]:
# 保存到 Excel
df_final.to_excel("spotify_genius_merged.xlsx", index=False)
print(f"Saved {len(df_final)} songs with lyrics.")

Saved 1049 songs with lyrics.
